In [15]:
from __future__ import print_function

import pandas as pd
import os
from dotenv import load_dotenv
from datetime import date, datetime, timedelta
import sqlalchemy
import pymysql
import openpyxl

In [16]:
load_dotenv()

(host, user, password, port, database) = (os.getenv('host'), os.getenv('user'), os.getenv('passwd'), os.getenv('port'), os.getenv('database'))

In [17]:
def get_connection():
    return sqlalchemy.create_engine(
        url="mysql+pymysql://{0}:{1}@{2}:{3}/{4}".format(
            user, password, host, port, database
        )
    )

In [18]:
engine = get_connection()

table = "cpt_codes"

In [19]:
cpt_path = 'ndcxls/PPRRVU23_JAN.xlsx'
start_row = 9
columns = ['code', 'description']
sqltypes_ndc = {'code': sqlalchemy.types.VARCHAR(length=8), 'description': sqlalchemy.types.VARCHAR(length=30)}

df = pd.read_excel(cpt_path, names=columns,usecols=[0,2], skiprows= start_row)
df.columns = columns

In [20]:
df.drop_duplicates(inplace=True)
duplicated = df.duplicated(subset=['code'])
df[df.duplicated(['code'], keep=False)]

,code,description


In [21]:
print(df.code.str.len().max())
print(df.description.str.len().max())

7
28.0


In [22]:
df.dtypes

code           object
description    object
dtype: object

In [23]:
df.to_sql(name=table,con=engine, if_exists='append', index=False, chunksize=100, dtype=sqltypes_ndc)

15856